# Analizando o avermelhamento interestelar e calculando fotometria sintética

## Autores

Kristen Larson, Lia Corrales, Stephanie T. Douglas, Kelle Cruz

Input from Emir Karamehmetoglu, Pey Lian Lim, Karl Gordon, Kevin Covey

## Tradução

Chrystian Luciano Pereira


## Objetivos de Aprendizagem
- Investigar a forma das curvas de extinção

- Deredden spectral energy distributions and spectra
- Calcular a extinção fotométrica e o avermelhamento
- Calcular a fotometria sintética para uma estrela avermelhada combinando `dust_extinction` e` synphot`
- Converter de frequência para comprimento de onda com equivalências do `astropy.unit`
- Plotagem com o `astropy.visualization`


## Palavras-chave
dust extinction, synphot, astroquery, units, fotometria, extinção, física, astronomia observacional

## Conteúdo complementar

* [Bessell & Murphy (2012)](https://ui.adsabs.harvard.edu/#abs/2012PASP..124..140B/abstract)



## Sumário

Nesse tutorial, veremos algumas curvas de extinção da literatura, usaremos uma dessas curvas para desavermelhar um espectro observado e praticaremos usando o fluxo de uma fonte de fundo para calcular as magnitudes de um modelo de extinção.

As principais bibliotecas que usaremos aqui [dust_extinction](https://dust-extinction.readthedocs.io/en/latest/) e [synphot](https://synphot.readthedocs.io/en/latest/), que são [pacotes afiliados ao Astropy](https://www.astropy.org/affiliated/). 

Recomendamos instalar os dois pacotes desta maneira:
```
pip install synphot
pip install dust_extinction
```

Esse tutorial necessita da versão v0.7 ou posterior da `dust_extinction`. Para garantir que todos os comandos funcionem corretamente, certifique-se de ter a versão correta instalada. Se você tiver a v0.6 ou anterior instalada, execute o seguinte comando para atualizar
```
pip install dust_extinction --upgrade
```

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import astropy.units as u
from astropy.table import Table
from dust_extinction.parameter_averages import CCM89, F99
from synphot import units, config
from synphot import SourceSpectrum,SpectralElement,Observation,ExtinctionModel1D
from synphot.models import BlackBodyNorm1D
from synphot.spectrum import BaseUnitlessSpectrum
from synphot.reddening import ExtinctionCurve
from astroquery.simbad import Simbad
from astroquery.mast import Observations
import astropy.visualization

# Introdução

A poeira no meio interestelar (ISM) reduz a luz das estrelas de fundo. A dependência da extinção com comprimento de onda é tal que a luz de comprimento de onda curto se extingue mais do que a luz de comprimento de onda longo. Chamamos esse efeito de *avermelhamento*.

Se você é novo na extinção, aqui está uma breve introdução aos tipos de quantidades envolvidas.
A mudança fracionária no fluxo da luz das estrelas é
$$
\frac{dF_\lambda}{F_\lambda} = -\tau_\lambda
$$

onde $\tau$ é a profundidade ótica e depende do comprimento de onda. Integrando ao longo da linha de visada, o fluxo resultante é uma função exponencial da profundidade ótica

$$
\tau_\lambda = -\ln\left(\frac{F_\lambda}{F_{\lambda,0}}\right).
$$

Para saber como definimos as magnitudes, geralmente mudamos a base de $e$ para 10,  
$$
\tau_\lambda = -2.303\log\left(\frac{F_\lambda}{F_{\lambda,0}}\right),
$$

e definir uma extinção $A_\lambda = 1.086 \,\tau_\lambda$ de modo que
$$
A_\lambda = -2.5\log\left(\frac{F_\lambda}{F_{\lambda,0}}\right).
$$


Existem duas mensagens básicas para levar para casa nesta derivação:

* A extinção introduz um fator de multiplicação $10^{-0.4 A_\lambda}$ ao fluxo;
* A extinção é definida em relação ao fluxo sem poeira, $F_{\lambda,0}$.


Assim que o astropy e os pacotes afiliados forem instalados, podemos importá-los conforme necessário:

# Exemplo 1: Investigando os modelos de Extinção

O pacote `dust_extinction` fornece vários modelos para extinção $ A_\lambda$ normalizado para $A_V$. As formas das curvas normalizadas são relativamente (e talvez surpreendentemente) uniformes na Via Láctea. A pequena variação que existe é frequentemente parametrizada pela razão de extinção ($A_V$) para avermelhamento no azul-visual ($E_{B-V}$), 

$$
R_V \equiv \frac{A_V}{E_{B-V}}
$$

onde $ E_{B-V} $ é a extinção diferencial $A_B-A_V$. Neste exemplo, mostramos a parametrização $ R_V $ para os modelos Clayton, Cardelli, & Mathis (1989, CCM) e Fitzpatrick (1999). [Mais opções de modelo estão disponíveis na documentação `dust_extinction`.](https://dust-extinction.readthedocs.io/en/latest/dust_extinction/model_flavors.html)

In [ ]:
# Criando uma matriz de comprimentos de onda
wav = np.arange(0.1, 3.0, 0.001)*u.micron

for model in [CCM89, F99]:
    for R in (2.0,3.0,4.0):
        # Inicializando o modelo de extinção
        ext = model(Rv=R)
        plt.plot(1/wav, ext(wav), label=model.name+' R='+str(R))
        
plt.xlabel('$\lambda^{-1}$ ($\mu$m$^{-1}$)')
plt.ylabel('A($\lambda$) / A(V)')
plt.legend(loc='best')
plt.title('Algumas Leis de Extinção')
plt.show()

Os astrônomos que estudam o meio interestelar frequentemente exibem curvas de extinção em função do inverso do comprimento de onda (número de onda) para mostrar a variação ultravioleta, como fazemos aqui. A extinção do infravermelho varia muito menos e se aproxima de zero no comprimento de onda longo na ausência de extinção independente do comprimento de onda.

# Exemplo 2: Desavermelhando um Espectro

Aqui nós desavermelhamos (desextinguímos) o espectro ultravioleta IUE e a fotometria óptica da estrela $\rho$ Oph (HD 147933).

Primeiro, usaremos astroquery para buscar o arquivo [espectro IUE do MAST](https://archive.stsci.edu/iue/):

In [ ]:
obsTable = Observations.query_object("HD 147933",radius="1 arcsec")
obsTable_spec=obsTable[obsTable['dataproduct_type']=='spectrum']
obsTable_spec.pprint()

obsids = obsTable_spec[39]['obsid']
dataProductsByID = Observations.get_product_list(obsids)
manifest = Observations.download_products(dataProductsByID)

Lemos os arquivos baixados em uma tabela astropy:

In [ ]:
t_lwr = Table.read('./mastDownload/IUE/lwr05639/lwr05639mxlo_vo.fits')
print(t_lwr)

A extensão `.quantity` nas próximas linhas irá ler as colunas da tabela em vetores de quantidade. Quantidades mantêm as unidades da coluna Tabela anexadas aos valores da matriz numpy.

In [ ]:
wav_UV = t_lwr['WAVE'][0,].quantity
UVflux = t_lwr['FLUX'][0,].quantity


Agora, usamos astroquery novamente para buscar a fotometria no Simbad para ir com o espectro IUE:

In [ ]:
custom_query = Simbad()
custom_query.add_votable_fields('fluxdata(U)','fluxdata(B)','fluxdata(V)')
phot_table=custom_query.query_object('HD 147933')
Umag=phot_table['FLUX_U']
Bmag=phot_table['FLUX_B']
Vmag=phot_table['FLUX_V']

Para converter a fotometria em fluxo, procuramos algumas [propriedades das bandas passantes fotométricas](http://ned.ipac.caltech.edu/help/photoband.lst), incluindo o fluxo de uma estrela de magnitude zero através de cada banda passante , também conhecido como ponto zero da banda passante.

In [ ]:
wav_U = 0.3660 * u.micron 
zeroflux_U_nu = 1.81E-23 * u.Watt/(u.m*u.m*u.Hz)
wav_B = 0.4400 * u.micron
zeroflux_B_nu = 4.26E-23 * u.Watt/(u.m*u.m*u.Hz)
wav_V = 0.5530 * u.micron
zeroflux_V_nu = 3.64E-23 * u.Watt/(u.m*u.m*u.Hz)

Os pontos zero que encontramos para as bandas ópticas não estão nas mesmas unidades que os fluxos IUE. Para piorar as coisas, os fluxos de ponto zero são $F_\nu$ e os fluxos IUE são $F_\lambda$. Para converter entre eles, o comprimento de onda é necessário. Felizmente, o astropy fornece uma maneira fácil de fazer a conversão com *equivalências*: 

In [ ]:
zeroflux_U = zeroflux_U_nu.to(u.erg/u.AA/u.cm/u.cm/u.s, 
                              equivalencies=u.spectral_density(wav_U))
zeroflux_B = zeroflux_B_nu.to(u.erg/u.AA/u.cm/u.cm/u.s, 
                              equivalencies=u.spectral_density(wav_B))
zeroflux_V = zeroflux_V_nu.to(u.erg/u.AA/u.cm/u.cm/u.s, 
                              equivalencies=u.spectral_density(wav_V))

Agora podemos converter de fotometria para fluxo usando a definição de magnitude:

$$
F=F_0\ 10^{-0.4\, m}
$$

In [ ]:
Uflux = zeroflux_U * 10.**(-0.4*Umag)
Bflux = zeroflux_B * 10.**(-0.4*Bmag)
Vflux = zeroflux_V * 10.**(-0.4*Vmag)

O uso dos vetores de quantidade do astropy nos permite tirar vantagem do astropy unit para os plots. [Chamar `astropy.visualization.quantity_support` explicitamente ativa o recurso.](Http://docs.astropy.org/en/stable/units/quantity.html#plotting-quantities)

Então, quando os objetos *quantity* são passados para as funções de plot do matplotlib, os rótulos dos eixos são automaticamente identificados com a unidade da quantidade. Além disso, as quantidades são convertidas automaticamente para as mesmas unidades ao combinar vários gráficos nos mesmos eixos.


In [ ]:
astropy.visualization.quantity_support()

plt.plot(wav_UV,UVflux,'m',label='UV')
plt.plot(wav_V,Vflux,'ko',label='U, B, V')
plt.plot(wav_B,Bflux,'ko')
plt.plot(wav_U,Uflux,'ko')
plt.legend(loc='best')
plt.ylim(0,3E-10)
plt.title('rho Oph')
plt.show()

Por fim, inicializamos o modelo de extinção, escolhendo os valores $ R_V = 5 $ e $ E_ {B-V} = 0,5 $. Esta estrela é famosa na comunidade por apresentar extinção e avermelhamento acentuados, devido a poeira e gás na linha de visada.

In [ ]:
Rv = 5.0  # Geralmente próximo a 3, mas cerca de 5 para essa estrela.
Ebv = 0.5
ext = F99(Rv=Rv)

Para extinguir (avermelhar) um espectro, multiplique pela função `ext.extinguish`. Para anular a extinção (desavermelhar), divida pelo mesmo `ext.extinguish`, como fazemos aqui:

In [ ]:
plt.semilogy(wav_UV,UVflux,'m',label='UV')
plt.semilogy(wav_V,Vflux,'ko',label='U, B, V')
plt.semilogy(wav_B,Bflux,'ko')
plt.semilogy(wav_U,Uflux,'ko')

plt.semilogy(wav_UV,UVflux/ext.extinguish(wav_UV,Ebv=Ebv),'b',
             label='dereddened: EBV=0.5, RV=5')
plt.semilogy(wav_V,Vflux/ext.extinguish(wav_V,Ebv=Ebv),'ro',
             label='dereddened: EBV=0.5, RV=5')
plt.semilogy(wav_B,Bflux/ext.extinguish(wav_B,Ebv=Ebv),'ro')
plt.semilogy(wav_U,Uflux/ext.extinguish(wav_U,Ebv=Ebv),'ro')

plt.legend(loc='best')
plt.title('rho Oph')
plt.show()

Observe que, ao diminuir o espectro, a característica de absorção em 2175 Angstrons é removida. Essa característica também pode ser vista como uma saliência proeminente nas curvas de extinção no Exemplo 1. O fato de termos removido suavemente a característica em 2175 Angstrons sugere que os valores que escolhemos, $R_V = 5$ e $E_{BV} = 0,5$, são um modelo razoável para a poeira do primeiro plano.

Os mais experientes com desavermelhamento devem notar que `dust_extinction` retorna $A_\lambda/A_V$, enquanto outras rotinas como o procedimento IDL fm_unred geralmente retornam $A_\lambda/E_ {BV}$ por padrão e precisam ser divididos por $R_V$ para permitir a comparação direta com `dust_extinction`.



# Exemplo 3: Cálculo do excesso de cor com o `synphot`

Calcular a extinção *fotométrica* de banda larga é mais difícil que pode parecer à primeira vista. Basta procurar $A_\lambda$ para uma banda passante específica? Nas condições certas, sim. Mas em geral, não.

Lembre-se que temos que integrar sobre uma banda passante para obter uma fotometria sintética,

$$
A = -2.5\log\left(\frac{\int W_\lambda F_{\lambda,0} 10^{-0.4A_\lambda} d\lambda}{\int W_\lambda F_{\lambda,0} d\lambda} \right),
$$

onde $W_\lambda$ é a fração da energia incidente que é transmitida através de um filtro. Ver o apêndice [Bessell & Murphy (2012)](https://ui.adsabs.harvard.edu/#abs/2012PASP..124..140B/abstract) para uma exelente revisão dos problemas e mal-entendidos comuns em fotometria sintética.

Um importante apontamento aqui, é  que a expressão acima não pode ser simplificada. A rigor, é impossível converter a extinção espectral $A_\lambda$ em um sistema de magnitudes sem conhecer a dependência do comprimento de onda do fluxo original da fonte  através do filtro em questão.

Um caso especial é assumir que o fluxo da fonte é contante na banda (ou seja, $F_\lambda = F$), então podemos cancelar esses fatores das integrais, e a extinção em magnitudes torna-se a média ponderada do fator de extinção em todo o filtro em questão. Nesse caso especial, $ A_\lambda$ em $\lambda_{\ rm eff}$ é uma boa aproximação para a extinção de magnitude.

Neste exemplo, vamos demonstrar o cálculo mais geral da extinção fotométrica. Usamos uma curva do corpo negro para o fluxo antes da poeira, aplicamos uma curva de extinção e realizamos a fotometria sintética para calcular a extinção e o avermelhamento em um sistema de magnitude.


Primeiro, vamos obter as curvas de transmissão do filtro:

In [ ]:
# Opcional, para quando o servidor de ftp STScI não estiver respondendo:
config.conf.vega_file='http://ssb.stsci.edu/cdbs/calspec/alpha_lyr_stis_008.fits'
config.conf.johnson_u_file='http://ssb.stsci.edu/cdbs/comp/nonhst/johnson_u_004_syn.fits'
config.conf.johnson_b_file='http://ssb.stsci.edu/cdbs/comp/nonhst/johnson_b_004_syn.fits'
config.conf.johnson_v_file='http://ssb.stsci.edu/cdbs/comp/nonhst/johnson_v_004_syn.fits'
config.conf.johnson_r_file='http://ssb.stsci.edu/cdbs/comp/nonhst/johnson_r_003_syn.fits'
config.conf.johnson_i_file='http://ssb.stsci.edu/cdbs/comp/nonhst/johnson_i_003_syn.fits'
config.conf.bessel_j_file='http://ssb.stsci.edu/cdbs/comp/nonhst/bessell_j_003_syn.fits'
config.conf.bessel_h_file='http://ssb.stsci.edu/cdbs/comp/nonhst/bessell_h_004_syn.fits'
config.conf.bessel_k_file='http://ssb.stsci.edu/cdbs/comp/nonhst/bessell_k_003_syn.fits'

u_band = SpectralElement.from_filter('johnson_u')
b_band = SpectralElement.from_filter('johnson_b')
v_band = SpectralElement.from_filter('johnson_v')
r_band = SpectralElement.from_filter('johnson_r')
i_band = SpectralElement.from_filter('johnson_i')
j_band = SpectralElement.from_filter('bessel_j')
h_band = SpectralElement.from_filter('bessel_h')
k_band = SpectralElement.from_filter('bessel_k')

Se você estiver executando isso com seu próprio python, consulte a [documentação do synphot](https://synphot.readthedocs.io/en/latest/#installation-and-setup) sobre como instalar sua própria cópia dos arquivos necessários.



A seguir, vamos fazer um fluxo de fundo ao qual aplicaremos a extinção. Aqui, fazemos um corpo negro de 10.000 K usando o mecanismo de modelo de dentro de `synphot` e normalizamos para $V$ = 10 no sistema de magnitude baseado em Vega.

In [ ]:
# Primeiro, crie um corpo negro a determinada temperatura.
sp = SourceSpectrum(BlackBodyNorm1D, temperature=10000)
# sp.plot(left=1, right=15000, flux_unit='flam', title='Blackbody')

# Obtenha o espectro de Vega como o fluxo do ponto zero.
vega = SourceSpectrum.from_vega()
# vega.plot(left=1, right=15000)

# Normalize o corpo negro para alguma magnitude escolhida, digamos V = 10.
vmag = 10.
v_band = SpectralElement.from_filter('johnson_v')
sp_norm = sp.normalize(vmag * units.VEGAMAG, v_band, vegaspec=vega)
sp_norm.plot(left=1, right=15000, flux_unit='flam', title='Normed Blackbody')

Agora inicializamos o modelo de extinção e escolhemos uma extinção de $A_V$ = 2. Para obter o modelo `dust_extinction` trabalhando com o `synphot`, criamos uma matriz de comprimento de onda e fazemos um elemento espectral com o modelo de extinção como uma tabela de consulta.


In [ ]:
# Inicialize o modelo de extinção e escolha a extinção, aqui Av = 2.
ext = CCM89(Rv=3.1)
Av = 2.

# Crie a matriz de comprimento de onda. 
wav = np.arange(0.1, 3, 0.001)*u.micron

# Faça um modelo de extinção no synphot usando uma 'lookup table'.
ex = ExtinctionCurve(ExtinctionModel1D, 
                     points=wav, lookup_table=ext.extinguish(wav, Av=Av))
sp_ext = sp_norm*ex
sp_ext.plot(left=1, right=15000, flux_unit='flam',
            title='Normed Blackbody with Extinction')

Fotometria sintética refere-se à modelagem de uma observação de uma estrela multiplicando o modelo teórico para o fluxo astronômico por meio de uma determinada função de resposta de filtro e, em seguida, integrando.

In [ ]:
# "Observe" a estrela através do filtro e integre-a para obter a magnitude fotométrica.
sp_obs = Observation(sp_ext, v_band)
sp_obs_before = Observation(sp_norm, v_band)
# sp_obs.plot(left=1, right=15000, flux_unit='flam',
#             title='Normed Blackbody with Extinction through V Filter')

Em seguida, `synphot` realiza a integração e calcula as magnitudes no sistema Vega.

In [ ]:
sp_stim_before = sp_obs_before.effstim(flux_unit='vegamag', vegaspec=vega)
sp_stim = sp_obs.effstim(flux_unit='vegamag', vegaspec=vega)
print('before dust, V =', np.round(sp_stim_before,1))
print('after dust, V =', np.round(sp_stim,1))

# Calculate extinction and compare to our chosen value.
Av_calc = sp_stim - sp_stim_before
print('$A_V$ = ', np.round(Av_calc,1))

Esta é uma boa verificação para fazermos. Normalizamos nosso espectro para $ V $ = 10 mag e adicionamos 2 mag de extinção visual, então o procedimento de fotometria sintética deve reproduzir esses valores escolhidos, e o faz. Agora estamos prontos para encontrar a extinção em outras bandas.

Calculamos a nova fotometria para os filtros Johnson e Bessell (infravermelho). Calculamos a extinção $A = \Delta m $ e o excesso de cor do gráfico, $ E (\lambda - V) = A_ \lambda - A_V $.

Observe que `synphot` calcula o comprimento de onda efetivo das observações para nós, o que é muito útil para traçar os resultados. Mostramos o avermelhamento com a curva de extinção do modelo para comparação no gráfico.

In [ ]:
bands = [u_band,b_band,v_band,r_band,i_band,j_band,h_band,k_band]

for band in bands:
    # Calcule a fotometria com a poeira:
    sp_obs = Observation(sp_ext, band, force='extrap')
    obs_effstim = sp_obs.effstim(flux_unit='vegamag', vegaspec=vega)
    # Calcule a fotometria sem a poeira:
    sp_obs_i = Observation(sp_norm, band, force='extrap')
    obs_i_effstim = sp_obs_i.effstim(flux_unit='vegamag', vegaspec=vega)
  
    # Extinction = mag with dust - mag without dust
    # Color excess = extinction at lambda - extinction at V
    color_excess = obs_effstim - obs_i_effstim - Av_calc
    plt.plot(sp_obs_i.effective_wavelength(), color_excess,'or')
    print(np.round(sp_obs_i.effective_wavelength(),1), ',', 
          np.round(color_excess,2))

# Plote a curva do modelo de extinção para comparação 
plt.plot(wav,Av*ext(wav)-Av,'--k')
plt.ylim([-2,2])
plt.xlabel('$\lambda$ (Angstrom)')
plt.ylabel('E($\lambda$-V)')
plt.title('Reddening of T=10,000K Background Source with Av=2')
plt.show()  

## Exercício
Tente alterar a temperatura do corpo negro para algo muito quente ou muito frio. Os valores de excesso de cor são iguais? Os comprimentos de onda efetivos mudaram?

Observe que a extinção fotométrica muda porque a transmissão do filtro não é uniforme. A taxa de transferência observada do filtro depende da forma do fluxo da fonte de fundo.